In [1]:
from env.constants import PICKUPABLE_OBJECTS, OPENABLE_OBJECTS, RECEPTACLE_OBJECTS, THOR_COMMIT_ID, SCENE_TYPE_TO_SCENES
from ai2thor.controller import Controller
from ai2thor.platform import CloudRendering

allowed_scenes = {
    "Kitchen": True,
    "LivingRoom": True,
    "Bedroom": True,
    "Bathroom": True,
}
all_scenes = []

for k, v in allowed_scenes.items():
    if v:
        all_scenes += SCENE_TYPE_TO_SCENES[k]

c = Controller(commit_id=THOR_COMMIT_ID, platform=CloudRendering)



thor-CloudRendering-eb93d0b6520e567bac8ad630462b5c0c4cea1f5f.zip: [   0%   0.0 s/B]  of 551.MB
thor-CloudRendering-eb93d0b6520e567bac8ad630462b5c0c4cea1f5f.zip: [   0%  98.4 KiB/s]  of 551.MB
thor-CloudRendering-eb93d0b6520e567bac8ad630462b5c0c4cea1f5f.zip: [   0% 162.8 KiB/s]  of 551.MB
thor-CloudRendering-eb93d0b6520e567bac8ad630462b5c0c4cea1f5f.zip: [   0% 252.7 KiB/s]  of 551.MB
thor-CloudRendering-eb93d0b6520e567bac8ad630462b5c0c4cea1f5f.zip: [   0% 366.1 KiB/s]  of 551.MB
thor-CloudRendering-eb93d0b6520e567bac8ad630462b5c0c4cea1f5f.zip: [   0% 612.0 KiB/s]  of 551.MB
thor-CloudRendering-eb93d0b6520e567bac8ad630462b5c0c4cea1f5f.zip: [   0% 743.0 KiB/s]  of 551.MB
thor-CloudRendering-eb93d0b6520e567bac8ad630462b5c0c4cea1f5f.zip: [   0% 820.7 KiB/s]  of 551.MB
thor-CloudRendering-eb93d0b6520e567bac8ad630462b5c0c4cea1f5f.zip: [   0% 884.5 KiB/s]  of 551.MB
thor-CloudRendering-eb93d0b6520e567bac8ad630462b5c0c4cea1f5f.zip: [   0% 938.4 KiB/s]  of 551.MB
thor-CloudRendering-eb93d0b6520e

In [10]:
from env.constants import OBJECT_TYPES_WITH_PROPERTIES, REARRANGE_SIM_OBJECTS, PICKUPABLE_OBJECTS_TO_EXISTING_SCENES
print(f'len(OBJECT_TYPES_WITH_PROPERTIES): {len(OBJECT_TYPES_WITH_PROPERTIES)}')
print(f'len(REARRANGE_SIM_OBJECTS): {len(REARRANGE_SIM_OBJECTS)}')
print(f'len(PICKUPABLE_OBJECTS_TO_EXISTING_SCENES): {len(PICKUPABLE_OBJECTS_TO_EXISTING_SCENES)}')
allf = [o for o, p in OBJECT_TYPES_WITH_PROPERTIES.items() if all([not t for t in p.values()])]
print(f'allf: {allf}')
print(f'len(allf): {len(allf)}')
seto = set(PICKUPABLE_OBJECTS + OPENABLE_OBJECTS + RECEPTACLE_OBJECTS)
print(f'len(seto): {len(seto)}')


len(OBJECT_TYPES_WITH_PROPERTIES): 118
len(REARRANGE_SIM_OBJECTS): 125
len(PICKUPABLE_OBJECTS_TO_EXISTING_SCENES): 62
allf: ['StoveKnob', 'Window', 'HousePlant', 'LightSwitch', 'Faucet', 'Curtains', 'GarbageBag', 'Painting', 'Television', 'FloorLamp', 'DeskLamp', 'Mirror', 'RoomDecor', 'Poster', 'Desktop', 'ShowerHead', 'ShowerGlass', 'VacuumCleaner']
len(allf): 18
len(seto): 100


In [12]:
import procthor.databases as db

assets = db._get_asset_database()
proc_obj = list(assets.keys())

In [13]:
l1 = [o for o in seto if o not in proc_obj]
print(f"{l1}, {len(l1)}")

['Shelf', 'ShowerDoor', 'StoveBurner', 'SinkBasin', 'Bathtub', 'Cabinet', 'Floor', 'BathtubBasin', 'Drawer'], 9


In [14]:
l2 = [o for o in proc_obj if o not in seto]
print(f"{l2}, {len(l2)}")

['Cart', 'ClothesDryer', 'DeskLamp', 'Desktop', 'Doorframe', 'Doorway', 'Faucet', 'FloorLamp', 'GarbageBag', 'HousePlant', 'LightSwitch', 'Painting', 'RoomDecor', 'ShowerHead', 'Television', 'VacuumCleaner', 'WashingMachine', 'Window'], 18


In [15]:
l3 = [o for o in l2 if o not in REARRANGE_SIM_OBJECTS]
print(f"{l3}, {len(l3)}")

['Cart', 'ClothesDryer', 'Doorframe', 'Doorway', 'WashingMachine'], 5


In [2]:
pick, open, recep = {}, {}, {}

for s in all_scenes:
    c.reset(scene=s)
    objs = c.last_event.metadata['objects']

    pick_b, open_b, recep_b = {}, {}, {}
    pick_n, open_n, recep_n = {}, {}, {}
    for pick_o in PICKUPABLE_OBJECTS:
        pick_b[pick_o] = False
        pick_n[pick_o] = 0
        for o in objs:
            if o['objectType'] == pick_o:
                pick_b[pick_o] = True
                pick_n[pick_o] += 1

    for open_o in OPENABLE_OBJECTS:
        open_b[open_o] = False
        open_n[open_o] = 0
        for o in objs:
            if o['objectType'] == open_o:
                open_b[open_o] = True
                open_n[open_o] += 1

    for recep_o in RECEPTACLE_OBJECTS:
        recep_b[recep_o] = False
        recep_n[recep_o] = 0
        for o in objs:
            if o['objectType'] == recep_o:
                recep_b[recep_o] = True
                recep_n[recep_o] += 1

    pick[s] = {
        'exist': pick_b,
        'entity': pick_n,
    }
    open[s] = {
        'exist': open_b,
        'entity': open_n,
    }
    recep[s] = {
        'exist': recep_b,
        'entity': recep_n,
    }

In [3]:
pick_bt, pick_nt, pick_st = {}, {}, {}
for ot in PICKUPABLE_OBJECTS:
    pick_bt[ot] = False
    pick_nt[ot] = 0
    pick_st[ot] = []
    for s in all_scenes:
        if pick[s]['exist'][ot]:
            pick_bt[ot] = True
            pick_nt[ot] += pick[s]['entity'][ot]
            pick_st[ot].append(s)

open_bt, open_nt, open_st = {}, {}, {}
for ot in OPENABLE_OBJECTS:
    open_bt[ot] = False
    open_nt[ot] = 0
    open_st[ot] = []
    for s in all_scenes:
        if open[s]['exist'][ot]:
            open_bt[ot] = True
            open_nt[ot] += open[s]['entity'][ot]
            open_st[ot].append(s)

recep_bt, recep_nt, recep_st = {}, {}, {}
for ot in RECEPTACLE_OBJECTS:
    recep_bt[ot] = False
    recep_nt[ot] = 0
    recep_st[ot] = []
    for s in all_scenes:
        if recep[s]['exist'][ot]:
            recep_bt[ot] = True
            recep_nt[ot] += recep[s]['entity'][ot]
            recep_st[ot].append(s)

In [5]:
for p in PICKUPABLE_OBJECTS:
    print(f"'{p}': {pick_nt[p]} {pick_st[p]},")

'AlarmClock': 30 ['FloorPlan301', 'FloorPlan302', 'FloorPlan303', 'FloorPlan304', 'FloorPlan305', 'FloorPlan306', 'FloorPlan307', 'FloorPlan308', 'FloorPlan309', 'FloorPlan310', 'FloorPlan311', 'FloorPlan312', 'FloorPlan313', 'FloorPlan314', 'FloorPlan315', 'FloorPlan316', 'FloorPlan317', 'FloorPlan318', 'FloorPlan319', 'FloorPlan320', 'FloorPlan321', 'FloorPlan322', 'FloorPlan323', 'FloorPlan324', 'FloorPlan325', 'FloorPlan326', 'FloorPlan327', 'FloorPlan328', 'FloorPlan329', 'FloorPlan330'],
'AluminumFoil': 3 ['FloorPlan21'],
'Apple': 30 ['FloorPlan1', 'FloorPlan2', 'FloorPlan3', 'FloorPlan4', 'FloorPlan5', 'FloorPlan6', 'FloorPlan7', 'FloorPlan8', 'FloorPlan9', 'FloorPlan10', 'FloorPlan11', 'FloorPlan12', 'FloorPlan13', 'FloorPlan14', 'FloorPlan15', 'FloorPlan16', 'FloorPlan17', 'FloorPlan18', 'FloorPlan19', 'FloorPlan20', 'FloorPlan21', 'FloorPlan22', 'FloorPlan23', 'FloorPlan24', 'FloorPlan25', 'FloorPlan26', 'FloorPlan27', 'FloorPlan28', 'FloorPlan29', 'FloorPlan30'],
'BaseballBa